In [ ]:
import numpy as np
from pythtb import *

In [ ]:
def fu_kane_mele(t, soc, m, beta):
    # set up Fu-Kane-Mele model
    lat = [[0, 1, 1], [1, 0, 1], [1, 1, 0]]
    orb = [[0, 0, 0], [0.25, 0.25, 0.25]]
    model = tb_model(3, 3, lat, orb, nspin=2)

    h = m*np.sin(beta)*np.array([1,1,1])
    dt = m*np.cos(beta)

    h0 = [0] + list(h)
    h1 = [0] + list(-h)

    model.set_onsite(h0, 0)
    model.set_onsite(h1, 1)

    # spin-independent first-neighbor hops
    for lvec in ([-1, 0, 0], [0, -1, 0], [0, 0, -1]):
        model.set_hop(t, 0, 1, lvec)

    model.set_hop(3*t + dt, 0, 1, [0, 0, 0], mode="add")

    # spin-dependent second-neighbor hops
    lvec_list = ([1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 1, 0], [0, -1, 1], [1, 0, -1])
    dir_list = ([0, 1, -1], [-1, 0, 1], [1, -1, 0], [1, 1, 0], [0, 1, 1], [1, 0, 1])
    for j in range(6):
        spin = np.array([0.]+dir_list[j])
        model.set_hop( 1j*soc*spin, 0, 0, lvec_list[j])
        model.set_hop(-1j*soc*spin, 1, 1, lvec_list[j])

    return model

In [ ]:
def compute_d4k_and_d2k(delta_k):
    # Compute d^4k as the determinant of the 4x4 difference matrix
    d4k = np.sqrt(np.linalg.det(delta_k @ delta_k.T))

    # Function to compute 2D plaquette area in 4D space
    def compute_plaquette_area(v1, v2):    
        """Compute the 2D plaquette area spanned by two 4D vectors in a non-orthogonal basis."""
        g = np.array([[np.dot(v1, v1), np.dot(v1, v2)],
                    [np.dot(v2, v1), np.dot(v2, v2)]])
        return np.sqrt(np.linalg.det(g))

    # Compute all unique plaquette areas
    plaquette_areas = {}
    for i in range(delta_k.shape[0]):
        for j in range(i + 1, delta_k.shape[0]):
            plaquette_areas[(i, j)] = compute_plaquette_area(delta_k[i], delta_k[j])

    return d4k, plaquette_areas

In [ ]:
t = 2.0  # spin-independent first-neighbor hop
soc = 1  # spin-dependent second-neighbor hop
m = 1  # magnetic field magnitude
beta = 0
fkm_model = fu_kane_mele(t, soc, m, beta)

lat_vecs = fkm_model.get_lat()  # shape: (vec, val)
recip_lat_vecs =  2 * np.pi * np.linalg.inv(lat_vecs).T  # shape: (vec, val)

dim_k = 3
dim_lam = 1
dim_total = dim_k + dim_lam

nks = 10, 10, 10
n_beta = 20
n_param = nks + (n_beta,)

max_beta = 2*np.pi
betas = np.linspace(0, max_beta, n_beta, endpoint=False) # change to endpoint=False for periodic

print(dim_k, dim_lam, dim_total)
print("Total number of parameter points: ", np.prod(n_param))
print(recip_lat_vecs)

In [ ]:
delta_q = np.zeros((dim_k, dim_total))  # (dim_k, dim_total)
delta_lam_red = np.zeros((dim_lam, dim_total))  # (dim_lam, dim_total)

# Construct delta_k with 4D embedding (extra zeros for adiabatic params)
np.fill_diagonal(delta_q[:, :dim_k], 1 / np.array(nks))

# Construct delta_lambda (embedding into 4D with zeros in k-space parts)
np.fill_diagonal(delta_lam_red[:, dim_k:], 1 / np.array(n_beta))

dq = np.vstack([delta_q, delta_lam_red])  # (dim_total, dim_total)

print("dq vectors: \n", dq)
print("det(dq) =", np.linalg.det(dq))

In [ ]:
# Construct delta_k with 4D embedding (extra zeros for adiabatic params)
bs = np.zeros((dim_k, dim_total))  # (3, dim_total)
bs[:, :dim_k] = recip_lat_vecs

# Construct delta_lambda (embedding into 4D with zeros in k-space parts)
beta_vec = np.zeros((dim_lam, dim_total))  # (dim_lam, dim_total)
np.fill_diagonal(beta_vec[:, dim_k:], max_beta)

basis_4space = np.vstack([bs, beta_vec])  # (dim_total, dim_total)

g4 = basis_4space @ basis_4space.T
sqrt_mtrc = np.sqrt(np.linalg.det(g4))

d4k = sqrt_mtrc * np.prod([np.linalg.norm(dq[i]) for i in range(dq.shape[0])])

print(f"basis vectors: \n {basis_4space}\n")

print(f"g = \n{g4}\n")

print("det(dk) =", np.linalg.det(basis_4space))
print(f"sqrt(det(g)) = {sqrt_mtrc}")
print(f"d^4k = {d4k}")

In [ ]:
d2k_planes = {}
metric_planes = {}
for i in range(4):
    for j in range(i+1, 4):
        stacked_basis = np.vstack([basis_4space[i], basis_4space[j]])
        stacked_q = np.vstack([dq[i], dq[j]])
        metric_2d = stacked_basis @ stacked_basis.T
        d2k = np.sqrt(np.linalg.det(metric_2d)) * np.linalg.norm(dq[i]) * np.linalg.norm(dq[j])

        d2k_planes[i,j] = d2k
        metric_planes[i,j] = metric_2d

for key, val in metric_planes.items():
    print(f"Metric for plane {key}: \n {val}")

for key, val in d2k_planes.items():
    print(f"d^2k_{key} = {val}")

In [ ]:
for key, value in d2ks.items():
    print(f"d^2k_{key} = {value}")

In [ ]:
# Construct delta_k with 4D embedding (extra zeros for adiabatic params)
delta_k = np.zeros((dim_k, dim_total))  # (3, dim_total)
delta_k[:, :dim_k] = recip_lat_vecs / np.array(nks)[:, None]

# Construct delta_lambda (embedding into 4D with zeros in k-space parts)
delta_lam = np.zeros((dim_lam, dim_total))  # (dim_lam, dim_total)
np.fill_diagonal(delta_lam[:, dim_k:], max_beta / np.array(n_beta))

dk = np.vstack([delta_k, delta_lam])  # (dim_total, dim_total)

print("dk vectors: \n",dk)
print("det(dk) =", np.linalg.det(dk))

In [ ]:
basis_4space = dk /np.linalg.norm(dk, axis=1, keepdims=True)
g4 = basis_4space @ basis_4space.T
sqrt_mtrc = np.sqrt(np.linalg.det(g4))
print(f"g = \n{g4}\n")
print(f"sqrt(det(g)) = {sqrt_mtrc}")

In [ ]:
delta_k = np.array([recip_lat_vecs[k_idx]/(nk) for k_idx, nk in enumerate(nks)]) 
dk1 = np.linalg.norm(recip_lat_vecs[0]/(nks[0]))
dk2 = np.linalg.norm(recip_lat_vecs[1]/(nks[1]))
dk3 = np.linalg.norm(recip_lat_vecs[2]/(nks[2]))
d_beta = max_beta/n_beta  # Step size for beta integration

d4k, d2ks = compute_d4k_and_d2k(dk) 
d3k = np.linalg.det(delta_k)
d3k_alt = np.prod([dk1, dk2, dk3])
d4k_alt = np.prod([dk1, dk2, dk3, d_beta])

print("d^4k (4D Volume Element):", d4k)
print("Plaquette Areas (d^2k):")
for key, value in d2ks.items():
    print(f"d^2k_{key} = {value}")

print()


print(f"dk1 = {dk1}, dk2 = {dk2}, dk3 = {dk3}, d_beta = {d_beta}")
print(f"dk1*dk2 = {dk1*dk2}")
print(f"dk1*dk2*dk3 = {d3k_alt}")
print(f"dk1*dk2*dk3*d_beta = {d4k_alt}")

print()

print(f"sqrt(g) = {sqrt_mtrc}")
print(f"dk1*dk2*dk3 * sqrt(g)  = {d3k_alt*sqrt_mtrc}")
print(f"dk1*dk2*dk3*d_beta * sqrt(g) = {d4k_alt*sqrt_mtrc}")
print(f"d3k = {d3k}")
print(f"d4k = {d4k}")

print()

# print(f"V_BZ / (Nk1 * Nk2 * Nk3) = {fkm_model.get_recip_vol()/np.prod(nks)}")

In [ ]:
0.03896363641360098 / 0.05061524843699827

In [ ]:
print(d2ks[0,1]*d2ks[2, 3])
print(d2ks[0,2]*d2ks[1, 3])
print(d2ks[0,3]*d2ks[1, 2])

In [ ]:
g12 = d2ks[0,1] / (dk1*dk2)
factor = (d4k/(d2ks[0,1]*d2ks[2, 3]))
print(g12)
print(factor)
print(factor*g12)